# Access Azure resources from an online endpoint with a user-assigned managed identity

Additional packages are required for this example including 
* azure.mgmt.msi
* azure.mgmt.storage

Install them with the following code:

In [4]:
# TODO: 
# %pip install azure-mgmt-msi
# %pip install azure-mgmt-storage
# %pip install azure-mgmt-authorization

## 1. Configure variables

### 1.1 Assign variables for the workspace and deployment

In [5]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

endpoint_name = "<ENDPOINT_NAME>"

In [6]:
# TODO: Delete

subscription_id = "6fe1c377-b645-4e8e-b588-52e57cc856b2"
resource_group = "role-creation-sample"
workspace_name = "test-workspace"

endpoint_name = "rolestestuai1"

### 1.2 Specify the storage details and file to access
A text file is located in the `managed-identities` folder called `hello.txt`. 

In [7]:
storage_account_name = "<STORAGE_ACCOUNT_NAME>"
storage_container_name = "<CONTAINER_TO_ACCESS>"
file_name = "<FILE_TO_ACCESS>"

In [8]:
# TODO: Delete

storage_account_name = "rolestestuai1"
storage_container_name = "blob1"
file_name = "hello.txt"

### 1.3 Decide on the name of your user identity:

In [9]:
uai_name = "<USER_ASSIGNED_IDENTITY_NAME>"

In [10]:
# TODO: 
uai_name = "uaitest777-1"

### 1.4 Retrieve the workspace location:

In [11]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    ManagedOnlineEndpoint,
    Model,
    CodeConfiguration,
    Environment,
)

credential = AzureCliCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

workspace_location = ml_client.workspaces.get(workspace_name).location

## 2. Configure deployment

### 2.1 Define an endpoint configuration

In [12]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key")

### 2.2 Define a deployment configuration

In [13]:
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=Model(path="../../model-1/model/"),
    code_configuration=CodeConfiguration(
        code="../../model-1/onlinescoring/", scoring_script="score_managedidentity.py"
    ),
    environment=Environment(
        conda_file="../../model-1/environment/conda.yml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1",
    ),
    instance_type="Standard_DS2_v2",
    instance_count=1,
    environment_variables={
        "STORAGE_ACCOUNT_NAME": storage_account_name,
        "STORAGE_CONTAINER_NAME": storage_container_name,
        "FILE_NAME": file_name,
        # We will update this after creating an identity
        "UAI_CLIENT_ID": "uai_client_id_place_holder",
    },
)

## 3. Create the managed identity

### 3.1 Get a handle to the `ManagedServiceIdentityClient`

In [14]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.msi import ManagedServiceIdentityClient
from azure.mgmt.msi.models import Identity

credential = DefaultAzureCredential()
msi_client = ManagedServiceIdentityClient(
    subscription_id=subscription_id,
    credential=credential,
)

### 3.2 Create the identity

In [16]:
msi_client.user_assigned_identities.create_or_update(
    resource_group_name=resource_group,
    resource_name=uai_name,
    parameters=Identity(location=workspace_location),
)

### 3.3 Retrieve the identity object

In [17]:
uai_identity = msi_client.user_assigned_identities.get(
    resource_group_name=resource_group,
    resource_name=uai_name,
)

## 4. Create storage account and container

### 4.1 Get a handle to the `StorageManagementclient`

In [18]:
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.mgmt.storage import StorageManagementClient
from azure.storage.blob import ContainerClient
from azure.mgmt.storage.models import Sku, StorageAccountCreateParameters, BlobContainer

credential = AzureCliCredential()
storage_client = StorageManagementClient(
    credential=credential, subscription_id=subscription_id
)

### 4.2 Define and create the storage account

In [19]:
storage_account_parameters = StorageAccountCreateParameters(
    sku=Sku(name="Standard_LRS"), kind="Storage", location=workspace_location
)

poller = storage_client.storage_accounts.begin_create(
    resource_group_name=resource_group,
    account_name=storage_account_name,
    parameters=storage_account_parameters,
)

poller.wait()

storage_account = poller.result()

### 4.3 Create a blob container

In [20]:
blob_container = storage_client.blob_containers.create(
    resource_group_name=resource_group,
    account_name=storage_account_name,
    container_name=storage_container_name,
    blob_container=BlobContainer(),
)

### 4.4 Get the storage account key and create a `ContainerClient`

In [21]:
res = storage_client.storage_accounts.list_keys(
    resource_group_name=resource_group,
    account_name=storage_account_name,
)
key = res.keys[0].value

container_client = ContainerClient(
    account_url=storage_account.primary_endpoints.blob,
    container_name=storage_container_name,
    credential=key,
)

### 4.5 Upload a blob to the container

In [22]:
file_path = "hello.txt"
with open(file_path, "rb") as f:
    container_client.upload_blob(name=file_name, data=f.read())

## 5. Create an online endpoint

In [23]:
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint)

In [24]:
# Update its identity
endpoint.identity = endpoint.identity.from_dict(
    {"type": "UserAssigned", "user_assigned_identities": {uai_identity.id: {}}}
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint)

In [25]:
endpoint.identity.as_dict()

{'principal_id': '00000000-0000-0000-0000-000000000000',
 'tenant_id': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'type': 'UserAssigned',
 'user_assigned_identities': {'/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourcegroups/role-creation-sample/providers/Microsoft.ManagedIdentity/userAssignedIdentities/uaitest777-1': {'principal_id': '5eb94664-dffa-4721-afcf-6a034f785eab',
   'client_id': '67dad48b-44af-4562-a235-1007cbe9924d'}}}

## 6. Give access permission to the managed identity

### 6.1 Get a handle to the AuthorizationManagementClient

In [26]:
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.authorization.models import (
    RoleAssignment,
    RoleDefinition,
    RoleAssignmentCreateParameters,
    RoleAssignmentProperties,
    RoleAssignmentPropertiesWithScope,
)
import uuid

auth_client = AuthorizationManagementClient(
    credential=credential, subscription_id=subscription_id
)

### 6.2 Get User-assigned identity details

In [27]:
uai_identity = msi_client.user_assigned_identities.get(
    resource_group_name=resource_group, resource_name=uai_name
)
uai_principal_id = uai_identity.principal_id
uai_client_id = uai_identity.client_id

### 6.3 Give permission to the user storage account

In [35]:
# TODO: 
role_name = "Storage Blob Data Reader"
scope = storage_account.id

role_defs = auth_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

In [37]:
role_def.id

'/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/providers/Microsoft.Authorization/roleDefinitions/2a2b9908-6ea1-4ae2-8e65-a410df84e7d1'

In [ ]:
role_name = "Storage Blob Data Reader"
scope = storage_account.id

role_defs = auth_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

auth_client.role_assignments.create(
    scope=scope,
    role_assignment_name=uuid.uuid4(),
    parameters=RoleAssignmentProperties(
        role_definition_id=role_def.id, principal_id=uai_principal_id
    ),
)

### 6.4 Retrieve the workspace and container registry objects

In [31]:
workspace = ml_client.workspaces.get(workspace_name)
container_registry = workspace.container_registry

### 6.5 Give permission to the container registry

In [ ]:
role_name = "AcrPull"
scope = container_registry

role_defs = auth_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

auth_client.role_assignments.create(
    scope=scope,
    role_assignment_name=uuid.uuid4(),
    parameters=RoleAssignmentProperties(
        role_definition_id=role_def.id, principal_id=uai_principal_id
    ),
)

### 6.6  Give permission to the workspace storage account

In [ ]:
role_name = "Storage Blob Data Reader"
scope = workspace.storage_account

role_defs = auth_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

auth_client.role_assignments.create(
    scope=scope,
    role_assignment_name=uuid.uuid4(),
    parameters=RoleAssignmentProperties(
        role_definition_id=role_def.id, principal_id=uai_principal_id
    ),
)

## 7. Create a deployment with your configuration

### 7.1 Update the deployment configuration with the UAI Client ID

In [ ]:
deployment.environment_variables["UAI_CLIENT_ID"] = uai_client_id

### 7.2 Create the deployment

In [ ]:
deployment = ml_client.online_deployments.begin_create_or_update(deployment)

### 7.3 Check the status of the deployment

In [ ]:
deployment.as_dict()

### 7.4 Get the deployment logs

In [ ]:
ml_client.online_deployments.get_logs(deployment.name, deployment.endpoint_name, 1000)

## 8. Confirm your endpoint deployed successfully

In [ ]:
sample_data = "../../model-1/sample-request.json"
ml_client.online_endpoints.invoke(endpoint_name=endpoint_name, request_file=sample_data)

## 9. Delete the endpoint and storage account

### 9.1 Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(endpoint_name)

### 9.2 Delete the storage account

In [ ]:
storage_client.storage_accounts.delete(
    resource_group_name=resource_group, account_name=storage_account_name
)